5年期定投 择时

In [ ]:
import pandas as pd
from datetime import datetime
import trdb2py
import numpy as np

isStaticImg = False
width = 960
height = 768

pd.options.display.max_columns = None
pd.options.display.max_rows = None

trdb2cfg = trdb2py.loadConfig('./trdb2.yaml')

In [ ]:
# 具体基金
asset = 'jqdata.000300_XSHG|1d'
# baselineasset = 'jrj.510310'
# asset = 'jrj.110011'
# baselineasset = 'jqdata.000300_XSHG|1d'

# 起始时间，0表示从最开始算起
tsStart = 0
tsStart = int(trdb2py.str2timestamp('2005-01-01', '%Y-%m-%d'))

# 结束时间，-1表示到现在为止
tsEnd = -1
tsEnd = int(trdb2py.str2timestamp('2020-12-31', '%Y-%m-%d'))

# 初始资金池
paramsinit = trdb2py.trading2_pb2.InitParams(
    money=10000,
)

# 买入参数，用全部的钱来买入（也就是复利）
paramsbuy = trdb2py.trading2_pb2.BuyParams(
    perHandMoney=1,
)

# # 买入参数，用全部的钱来买入（也就是复利）
# paramsbuy2 = trdb2py.trading2_pb2.BuyParams(
#     perHandMoney=1/6,
# )

# 买入参数，用全部的钱来买入（也就是复利）
paramsbuy2 = trdb2py.trading2_pb2.BuyParams(
    perHandMoney=0.5,
)

# 卖出参数，全部卖出
paramssell = trdb2py.trading2_pb2.SellParams(
    perVolume=1,
)

paramsaip = trdb2py.trading2_pb2.AIPParams(
    money=10000,
    type=trdb2py.trading2_pb2.AIPTT_MONTHDAY,
    day=1,
)

# 止盈参数，120%止盈
paramstakeprofit = trdb2py.trading2_pb2.TakeProfitParams(
    perVolume=1,
#     isOnlyProfit=True,
#     isFinish=True,
)

# 止盈参数，120%止盈
paramstakeprofit2 = trdb2py.trading2_pb2.TakeProfitParams(
    perVolume=1,
#     isOnlyProfit=True,
    isFinish=True,
)

# 止盈参数，120%止盈
paramstakeprofit1 = trdb2py.trading2_pb2.TakeProfitParams(
    perVolume=1,
    isOnlyProfit=True,
#     isFinish=True,
)

# 止盈参数，120%止盈
paramstakeprofit3 = trdb2py.trading2_pb2.TakeProfitParams(
    perVolume=1,
    isOnlyProfit=True,
    isFinish=True,
)

# 卖出参数，全部卖出
paramssell7 = trdb2py.trading2_pb2.SellParams(
#     perVolume=1,
    keepTime=7 * 24 * 60 * 60,
)

lststart = [1, 2, 3, 4, 5]
lsttitle = ['周一', '周二', '周三', '周四', '周五']

In [ ]:
def calcweekday2val2(wday, offday):
    if offday == 1:
        if wday == 5:
            return 3
    if offday == 2:
        if wday >= 4:
            return 4
    if offday == 3:
        if wday >= 3:
            return 5
    if offday == 4:
        if wday >= 2:
            return 6
        
    return offday


def getAIPLastTs(pnl):
    ctrlnums = len(pnl.lstCtrl)

    if ctrlnums <= 0:
        return -1
    
    if pnl.lstCtrl[ctrlnums - 1].type == trdb2py.trading2_pb2.CTRL_SELL:
        return pnl.lstCtrl[ctrlnums - 1].ts
    
    return -1

def getLastResult(pnl) -> dict:
    nums = len(pnl.values)

    if nums <= 0:
        return None
    
    return {'cost': pnl.values[nums - 1].cost, 'value': pnl.values[nums - 1].value}

def getLastCtrl(pnl):
    ctrlnums = len(pnl.lstCtrl)

    if ctrlnums <= 0:
        return None
    
    return pnl.lstCtrl[ctrlnums - 1]


def getPNLValueWithTimestamp(ts, pnl: trdb2py.trading2_pb2.PNLAssetData) -> int:
    for i in range(0, len(pnl.values)):
        if ts == pnl.values[i].ts:
            return i

        if ts < pnl.values[i].ts:
            pnl.values.insert(i, trdb2py.trading2_pb2.PNLDataValue(ts=ts))

            return i

    pnl.values.append(trdb2py.trading2_pb2.PNLDataValue(ts=ts))

    return len(pnl.values) - 1

def mergePNL(lstpnl: list) -> trdb2py.trading2_pb2.PNLAssetData:
    pnl = trdb2py.trading2_pb2.PNLAssetData()

    for vpnl in lstpnl:
        v = vpnl['pnl']

        for cai in range(0, len(v.values)):
            di = getPNLValueWithTimestamp(v.values[cai].ts, pnl)
            pnl.values[di].value += v.values[cai].value
            pnl.values[di].cost += v.values[cai].cost

            if pnl.values[di].cost > 0:
                pnl.values[di].perValue = pnl.values[di].value / \
                    pnl.values[di].cost
            else:
                pnl.values[di].perValue = 1
                
#             pnl.values[di].perValue = 2

    return pnl

def getNextMonthDay1(ts):
    dt = datetime.utcfromtimestamp(ts)
    
    if dt.month == 12:
        return datetime(dt.year + 1, 1, 1).timestamp()
    
    return datetime(dt.year, dt.month + 1, 1).timestamp()


def rmPNLValuesWithTimestamp(ts, pnl: trdb2py.trading2_pb2.PNLAssetData):
    i = getPNLValueWithTimestamp(ts, pnl)
    pnl.values.extend(pnl.values[0:i+1])
    
    
def getPNLTimestampLowInMonth(pnl: trdb2py.trading2_pb2.PNLAssetData) -> list:
    ts = 0
    dt = None
    lastPerValue = 0
    arr = []

    for i in range(0, len(pnl.values)):
        v = pnl.values[i]

        if ts == 0:
            ts = v.ts
            dt = datetime.utcfromtimestamp(ts)
            lastPerValue = v.perValue
        else:
            cdt = datetime.utcfromtimestamp(v.ts)
            if dt.year == cdt.year and dt.month == cdt.month:
                if lastPerValue > v.perValue:
                    ts = v.ts
                    dt = cdt
                    lastPerValue = v.perValue

                if i == len(pnl.values) - 1:
                    arr.append(ts)
            else:
                arr.append(ts)

                ts = v.ts
                dt = cdt
                lastPerValue = v.perValue

    return arr


def getPNLTimestampHighInMonth(pnl: trdb2py.trading2_pb2.PNLAssetData) -> list:
    ts = 0
    dt = None
    lastPerValue = 0
    arr = []

    for i in range(0, len(pnl.values)):
        v = pnl.values[i]

        if ts == 0:
            ts = v.ts
            dt = datetime.utcfromtimestamp(ts)
            lastPerValue = v.perValue
        else:
            cdt = datetime.utcfromtimestamp(v.ts)
            if dt.year == cdt.year and dt.month == cdt.month:
                if lastPerValue < v.perValue:
                    ts = v.ts
                    dt = cdt
                    lastPerValue = v.perValue

                if i == len(pnl.values) - 1:
                    arr.append(ts)
            else:
                arr.append(ts)

                ts = v.ts
                dt = cdt
                lastPerValue = v.perValue

    return arr

In [ ]:
asset = 'jrj.110011'
# asset = 'jqdata.000036_XSHG|1d'
# asset = 'jqdata.000032_XSHG|1d'
asset = 'jqdata.000300_XSHG|1d'

# baseline    
s0 = trdb2py.trading2_pb2.Strategy(
    name="normal",
    asset=trdb2py.str2asset(asset),         
)
        
buy0 = trdb2py.trading2_pb2.CtrlCondition(
    name='buyandhold',
)

paramsbuy = trdb2py.trading2_pb2.BuyParams(
    perHandMoney=1,
)

paramsinit = trdb2py.trading2_pb2.InitParams(
    money=10000,
)

s0.buy.extend([buy0])
s0.paramsBuy.CopyFrom(paramsbuy)
s0.paramsInit.CopyFrom(paramsinit)        
p0 = trdb2py.trading2_pb2.SimTradingParams(
    assets=[trdb2py.str2asset(asset)],
    startTs=tsStart,
    endTs=tsEnd,
    strategies=[s0],
    title='沪深300',
)  

pnlBaseline = trdb2py.simTrading(trdb2cfg, p0)
trdb2py.showPNL(pnlBaseline, toImg=isStaticImg, width=width, height=height)

In [ ]:
lstparams = []

for cy in range(2005, 2016):
    for cm in range(1, 13):
        tsStart = int(trdb2py.str2timestamp('{}-{}-01'.format(cy, cm), '%Y-%m-%d'))
        tsEnd = int(trdb2py.str2timestamp('{}-{}-01'.format(cy + 5, cm), '%Y-%m-%d'))

        buy0 = trdb2py.trading2_pb2.CtrlCondition(
            name='monthdayex',
            vals=[1],
        )

        s0 = trdb2py.trading2_pb2.Strategy(
            name="normal",
            asset=trdb2py.str2asset(asset),
        )

    #     paramsaip = trdb2py.trading2_pb2.AIPParams(
    #         money=10000,
    #         type=trdb2py.trading2_pb2.AIPTT_WEEKDAY,
    #         day=1,
    #     )

        s0.buy.extend([buy0])
        s0.paramsBuy.CopyFrom(paramsbuy)
        s0.paramsSell.CopyFrom(paramssell)
    #     s0.paramsInit.CopyFrom(paramsinit)
        s0.paramsAIP.CopyFrom(paramsaip)
        lstparams.append(trdb2py.trading2_pb2.SimTradingParams(
            assets=[trdb2py.str2asset(asset)],
            startTs=tsStart,
            endTs=tsEnd,
            strategies=[s0],
            title='{}-{}定投5年'.format(cy, cm),
        ))
 
lstaippnl = trdb2py.simTradings(trdb2cfg, lstparams)

trdb2py.showPNLs(lstaippnl + [pnlBaseline], toImg=isStaticImg, width=width, height=height)

In [ ]:
dfpnl1b = trdb2py.buildPNLReport(lstaippnl + [pnlBaseline])

dfpnl1b[['title', 'maxDrawdown', 'maxDrawdownStart', 'maxDrawdownEnd', 'totalReturns', 'sharpe', 'annualizedReturns', 'annualizedVolatility', 'variance']].sort_values(by='totalReturns', ascending=False)